In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.preprocessing import LabelEncoder
from itertools import product


import time

In [37]:
test = pd.read_csv('../data/test.csv', index_col=0)
submit = pd.read_csv('../data/sample_submission.csv')
data = pd.read_pickle('data.pkl')


In [38]:
data.columns

Index(['month_idx', 'shop_id', 'item_id', 'item_cnt_month', 'cat_id',
       'type_code', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2',
       'item_cnt_month_lag_3', 'item_cnt_month_lag_6', 'delta_price_lag',
       'delta_revenue_lag_1', 'month', 'days', 'item_last_sale',
       'item_shop_first_sale', 'item_first_sale'],
      dtype='object')

In [40]:
data = data[[
'month_idx', 'shop_id', 'item_id', 'item_cnt_month', 'cat_id',
       'type_code', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2',
       'item_cnt_month_lag_3', 'item_cnt_month_lag_6',
       'delta_price_lag', 'delta_revenue_lag_1', 'month', 'days',
       'item_last_sale', 'item_shop_first_sale', 'item_first_sale'
]]


In [41]:
data.month_idx.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 23], dtype=int8)

In [42]:
X_train = data[data.month_idx < 21].drop(['item_cnt_month'], axis=1)
y_train = data[data.month_idx < 21]['item_cnt_month']
X_valid = data[data.month_idx == 21].drop(['item_cnt_month'], axis=1)
y_valid  = data[data.month_idx == 21]['item_cnt_month']
X_test = data[data.month_idx ==23].drop(['item_cnt_month'], axis=1)


In [43]:
del data

In [44]:
import gc


In [45]:
gc.collect()

160

In [46]:

ts = time.time()

model = XGBRegressor(
    max_depth=8,
    n_estimators = 1000,
    min_child_weight = 300,
    colsample_bytree=0.8,
    eta=0.3,
    seed=42
)

model.fit(X_train, y_train,eval_metric="rmse", eval_set=[(X_train, y_train),(X_valid, y_valid)],
    verbose=True,
    early_stopping_rounds=10)

time.time() - ts


[0]	validation_0-rmse:2.65660	validation_1-rmse:3.04976
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:2.46676	validation_1-rmse:2.89841
[2]	validation_0-rmse:2.36707	validation_1-rmse:2.80649
[3]	validation_0-rmse:2.31106	validation_1-rmse:2.75849
[4]	validation_0-rmse:2.27623	validation_1-rmse:2.73639
[5]	validation_0-rmse:2.25695	validation_1-rmse:2.72322
[6]	validation_0-rmse:2.24045	validation_1-rmse:2.70743
[7]	validation_0-rmse:2.22801	validation_1-rmse:2.68905
[8]	validation_0-rmse:2.21902	validation_1-rmse:2.68155
[9]	validation_0-rmse:2.21260	validation_1-rmse:2.67034
[10]	validation_0-rmse:2.20627	validation_1-rmse:2.66837
[11]	validation_0-rmse:2.20249	validation_1-rmse:2.66724
[12]	validation_0-rmse:2.19651	validation_1-rmse:2.66645
[13]	validation_0-rmse:2.19339	validation_1-rmse:2.67195
[14]	validation_0-rmse:2.18968	validation_1-rmse:2.67

37.726271629333496

In [47]:
y_pred = model.predict(X_valid)
y_test = model.predict(X_test)

submission = pd.DataFrame({
    "ID": test.index,
    "item_cnt_month": y_test
})

submission.to_csv('xgb_submission.csv', index=False )


In [17]:
submit

,0,100
0,1,100
1,2,100
2,3,100
3,4,100
4,5,100
...,...,...
3054,3055,100
3055,3056,100
3056,3057,100
3057,3058,100


In [16]:
submission

,ID,item_cnt_month
0,0,0.360681
1,1,0.000000
2,2,0.383577
3,3,1.242619
4,4,2.760417
...,...,...
3055,3055,0.765410
3056,3056,0.000894
3057,3057,0.000894
3058,3058,0.000894
